In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("CUDA Available:", torch.cuda.is_available())

CUDA Available: False


In [2]:
import torch

# Creating a tensor from a list
t1 = torch.tensor([1, 2, 3])
print(t1)

# Creating a tensor with predefined values
t2 = torch.zeros(3, 3)  # 3x3 matrix of zeros
t3 = torch.ones(2, 4)   # 2x4 matrix of ones
t4 = torch.rand(2, 2)   # 2x2 matrix of random values between 0 and 1

print(t2)
print(t3)
print(t4)

tensor([1, 2, 3])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[0.9899, 0.0761],
        [0.7127, 0.7101]])


In [3]:
t = torch.rand(3, 4)

print(f"Shape: {t.shape}")  # Dimensions of the tensor
print(f"Data type: {t.dtype}")  # Data type (default is float32)
print(f"Device: {t.device}")  # CPU or GPU

Shape: torch.Size([3, 4])
Data type: torch.float32
Device: cpu


In [4]:
x = torch.tensor([[1, 2], [3, 4]])
y = torch.tensor([[5, 6], [7, 8]])

# Element-wise operations
print(x + y)  # Addition
print(x * y)  # Multiplication
print(torch.sqrt(x.float()))  # Square root (requires float type)

# Matrix multiplication
print(x @ y)  # Equivalent to torch.matmul(x, y)

# Reshaping tensors
z = torch.arange(6).reshape(2, 3)
print(z)


tensor([[ 6,  8],
        [10, 12]])
tensor([[ 5, 12],
        [21, 32]])
tensor([[1.0000, 1.4142],
        [1.7321, 2.0000]])
tensor([[19, 22],
        [43, 50]])
tensor([[0, 1, 2],
        [3, 4, 5]])


In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU
    x = x.to(device)
    print(f"Tensor is now on: {x.device}")
else:
    print("CUDA is not available. Running on CPU.")

CUDA is not available. Running on CPU.


In [6]:
import torch
from torch.utils.data import Dataset

class SimpleDataset(Dataset):
    def __init__(self, N=100):
        self.x = torch.linspace(-2, 2, N)
        self.y = 2 * self.x + 3  # Linear function

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Create dataset instance
dataset = SimpleDataset(N=200)

# Fetch a single data point
idx = 10
x_sample, y_sample = dataset[idx]
print(f"x: {x_sample}, y: {y_sample}")

x: -1.798995018005371, y: -0.5979900360107422


In [7]:
from torch.utils.data import DataLoader

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Iterate through batches
for batch in dataloader:
    x_batch, y_batch = batch
    print(f"Batch - x: {x_batch}, y: {y_batch}")

    break # For site impagination

Batch - x: tensor([-0.8744, -1.2161]), y: tensor([1.2513, 0.5678])


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load dataset from sklearn
data = fetch_california_housing()
X, y = data.data, data.target

In [9]:
# Standardize features for better training stability
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

y = y.reshape(-1, 1)  # Reshape target to be a column vector

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [10]:
class CaliforniaHousingDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create Dataset instances
train_dataset = CaliforniaHousingDataset(X_train_tensor, y_train_tensor)
test_dataset = CaliforniaHousingDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [11]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.fc2 = nn.Linear(hidden_size, output_size) # Output layer

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))  # Apply ReLU activation to first layer
        x = self.fc2(x)          # Output layer (no activation for now)
        return x

# Create an instance of the model
model = SimpleNN(input_size=8, 
                 hidden_size=64, 
                 output_size=1)
print(model)

SimpleNN(
  (fc1): Linear(in_features=8, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [12]:
# Sample data from the dataset
x_batch, y_batch = next(iter(train_loader))

# Check the shape of the batch
print(f"Shape of x_batch: {x_batch.shape}. Shape of y_batch: {y_batch.shape}")

# Forward pass through the model
y_prediction = model(x_batch)

# Visualizing a value compared to the real (expected) solution
print(f"Real value: {y_batch[0].item()}. Model prediction: {y_prediction[0].item()}.")

Shape of x_batch: torch.Size([16, 8]). Shape of y_batch: torch.Size([16, 1])
Real value: 3.6110000610351562. Model prediction: 0.283534973859787.


In [13]:
import torch

# Create a leaf tensor
x = torch.linspace(0, 1, 20, requires_grad=True)

# Compute y = x**2
y = torch.square(x)

# Compute loss = sum(x**2)
loss = torch.sum(y)

# Compute gradient of the loss
loss.backward()

# Extract gradient wrt x -> d/dx loss(x^**2) = 2*x
g = x.grad
print(g)

tensor([0.0000, 0.1053, 0.2105, 0.3158, 0.4211, 0.5263, 0.6316, 0.7368, 0.8421,
        0.9474, 1.0526, 1.1579, 1.2632, 1.3684, 1.4737, 1.5789, 1.6842, 1.7895,
        1.8947, 2.0000])


In [14]:
# Define loss function (for example, MSE)
loss_fn = nn.MSELoss()

# Define optimizer (feeding the model parameters into it)
# Adam -> variant of SGD algorithm commonly used nowadays
#   lr -> "learning rate"
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

# Set other parameters (e.g. the number of epochs: number of times the training loop is repeated)
n_epochs = 50

# Epoch cycle
for epoch in range(n_epochs):
    avg_loss = 0.0

    # Training loop
    for k, data in enumerate(train_loader):
        # Get x, y from data
        x, y = data

        # Send to device
        x = x.to(device)
        y = y.to(device)

        # Compute neural network prediction
        y_pred = model(x)

        # Compare y_pred with the real y
        loss = loss_fn(y_pred, y)

        # Compute gradient
        loss.backward()

        # Update model weights
        optimizer.step()
        optimizer.zero_grad() # Reset the optimizer state: IMPORTANT

        # Print out the avg value of the loss
        # Commented for site impagination
        # print(f"Epoch: {epoch}. Avg Loss: {loss.item() / (k+1):0.4f}", end="\r")
    # print()

# Saving the model after the cycle
torch.save(model.state_dict(), "path-for-model.pth")

In [15]:
# Disable gradient memorization
with torch.no_grad():
    # Sample data from the dataset
    x_batch, y_batch = next(iter(train_loader))

    # Send to device
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    # Forward pass through the model
    y_prediction = model(x_batch)

    print(f"Prediction: {y_prediction[0].item():0.4f}. True: {y_batch[0].item():0.4f}.")

Prediction: 3.2548. True: 3.9080.
